# Scrape Data

1. Import Selenium and BeautifulSoup
2. Open Facebook.com and login
3. Go the the requested facebook page
4. Click all "See More" Buttons on the page to show the full text
5. Extract all posts from the page using beautifulsoup

In [64]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [87]:
#### Specify vairables here ####

# Path to chromedriver
chromedriver_path = r'C:\Users\aseem\chromedriver.exe'
url = "https://www.facebook.com/groups/1541028822851688/search/?q=rejected"
# Number of times to scroll down
scrolls = 30

# Facebook login credentials
email = os.environ.get('facebook_email')
f_password = os.environ.get('facebook_password')

### Opening Facebook.com and Signing In!

In [66]:
### Ignoring warnings ###
import warnings
warnings.filterwarnings('ignore')

# Prevents notifications from popping up
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome(chromedriver_path, chrome_options=chrome_options)

#open the webpage
driver.get("http://www.facebook.com")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys(email)
password.clear()
password.send_keys(f_password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='login']"))).click()

print("We are logged in!!")

We are logged in!!


### Opening the specified url and scrolling to the end of the page

In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# opening the specified url
driver.get(url)
time.sleep(2)

# loop to scroll down and click on "See more" buttons until the bottom of the page is reached
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # Try to find 'See more' buttons and click them
    see_more_buttons = driver.find_elements(By.XPATH, "//div[contains(text(), 'See more')]")
    for button in see_more_buttons:
        try:
            action = ActionChains(driver)
            action.move_to_element(button).perform()
            button.click()
            time.sleep(.5)
        except:
            pass

    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


In [83]:
# scroll to the top of the page
driver.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

### Extract posts using BeautifulSoup

In [90]:
soup = BeautifulSoup(driver.page_source, "html.parser")
all_posts = soup.find_all("div", {"class": "x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z"})

In [91]:
post_list = []
for i in range(len(all_posts)):
    plain_text = (all_posts[i].get_text(separator="\n").rsplit("Shared with Members of Pakistani Students in USA"))
    post_list.append(plain_text[1].rsplit("All reactions")[0])
    
print(f"The number of posts extracted: {len(post_list)}")

The number of posts extracted: 99


### Save the Data to csv

In [92]:
df = pd.DataFrame(post_list, columns=['Text'])
df.to_csv('data/accepted_post_list.csv')